In [1]:
import os
import re
import datetime
import copy

import numpy as np
import pandas as pd
from pandas import DataFrame, Series
from tqdm.auto import tqdm
from pandarallel import pandarallel

tqdm.pandas()
pandarallel.initialize(progress_bar=False)
# Сброс ограничений на число столбцов
pd.set_option("display.max_columns", 200)

# Сброс ограничений на количество символов в записи
pd.set_option("display.max_colwidth", 200)

INFO: Pandarallel will run on 32 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.


/home/gleb/.cache/pypoetry/virtualenvs/contract-Fr7eyqb1-py3.10/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Что нужно сделать: 
- добавить удаление плохих символов в get_num 
- удалить location из parsing_org
- добавиьт колонку с type_code

Что надо сделать:
- обработка данных организаций
    - Обаработать даты
    - Убрать organization_level, добавить уровень бюджета


- заполнение бюджетного уровня + (найти данные за 2014 год по кодам видов расходов)
- адресов заказчиков (найти плохие адреса и тоже их поменять) +
- дополнение адресов поставщиков (теже что по инн и ближайшие по времени)
- расшифровка кбк +  
- расшифровка адресов 
- обработка дат +

## Обработка данных организаций

In [13]:
df = pd.read_csv("../data/raw_data/org/2014/0.csv", sep="|", dtype="str")
df.shape

(4499, 35)

In [3]:
pd.DataFrame(df.isnull().sum())

,0
code,0
code_type,0
access_blocking,4499
full_name,0
short_name,268
adress,0
code_registr,45
date_registration,0
date_last_change,255
inn,0


### Обработка дат

In [4]:
df[["date_registration", "date_last_change", "date_registration_tax", "date_iky"]]

,date_registration,date_last_change,date_registration_tax,date_iky
0,02.12.2016,20.07.2021 22:45:35,16.09.2010,36423235526642301001
1,22.07.2016,05.08.2022 03:17:01,01.07.1990,24909004514490901001
2,18.11.2016,04.07.2023 07:18:30,16.09.1996,35052008899505001001
3,25.07.2016,24.01.2023 15:29:08,14.10.1992,27801033297780101001
4,19.07.2016,20.06.2022 16:14:31,10.05.2017,35721001232572101001
...,...,...,...,...
4494,21.07.2016,06.12.2022 10:52:30,24.10.2022,37012003880701201001
4495,19.12.2016,10.12.2021 17:02:30,23.08.2021,35053033947505301001
4496,13.12.2016,15.03.2023 19:07:51,28.03.2022,33024000502302301001
4497,31.12.2010,NaN,29.10.1999,38901010312890101001


In [15]:
daict_month = {
    "января": "01",
    "февраля": "02",
    "марта": "03",
    "апреля": "04",
    "мая": "05",
    "июня": "06",
    "июля": "07",
    "августа": "08",
    "сентября": "09",
    "октября": "10",
    "ноября": "11",
    "декабря": "12",
}


def date_extract(date: str, uniq_number):
    date_copy = date
    if not date or date == '--.--.----':
        return None

    date = date.replace("Загрузка ...", "").strip()
    try:
        return datetime.datetime.strptime(date, "%d.%m.%Y").date()
    except ValueError:
        pass

    try:
        return datetime.datetime.strptime(date[:10], "%d.%m.%Y").date()
    except ValueError:
        pass

    try:
        return datetime.datetime.strptime(date.split()[0], "%d.%m.%Y").date()
    except ValueError:
        pass

    for key, value in daict_month.items():
        if key in date:
            date = date.replace(key, value)
            date = ".".join(date.split())

    date = date.split(".")

    if len(date) == 2:
        date = ".".join(["01"] + date)
        return datetime.datetime.strptime(date[:10], "%d.%m.%Y").date()
    # логи
    print(uniq_number)
    print(date)

In [16]:
def processing_date_org(df: Series, columns: list):
    dict_result = {}
    # если мы получает nan при обращении к дате в DataFrame, то
    # np.nan == nan возращает False, однако у nan type float, когда у всех дат str
    uniq_number = df["code"]

    date_registration = df["date_registration"] if type(df["date_registration"]) == str else None
    date_last_change = df["date_last_change"] if type(df["date_last_change"]) == str else None
    date_registration_tax = (
        df["date_registration_tax"] if type(df["date_registration_tax"]) == str else None
    )
    date_iky = df["date_iky"] if type(df["date_iky"]) == str else None

    dict_result["date_registration"] = date_extract(date_registration, uniq_number)
    dict_result["date_last_change"] = date_extract(date_last_change, uniq_number)
    dict_result["date_registration_tax"] = date_extract(date_registration_tax, uniq_number)
    #dict_result["date_iky"] = date_extract(date_iky, uniq_number)

    return pd.Series(dict_result)[columns].to_list()

In [17]:
df_copy = df.copy()

In [18]:
df_copy = df.copy()

In [19]:
df.loc[df.code == '03113003261', ["date_registration", "date_last_change", "date_registration_tax"]]

,date_registration,date_last_change,date_registration_tax
3885,--.--.----,NaN,NaN


In [21]:
columns = ["date_registration", "date_last_change", "date_registration_tax"]
for file_name in tqdm(
    sorted(os.listdir("../data/raw_data/org/2014/"), key=lambda x: int(x.removesuffix(".csv")))
):
    df_now = pd.read_csv(
        os.path.join("../data/raw_data/org/2014/", file_name), sep="|", dtype="str"
    )
    df_now[columns] = df_now.parallel_apply(
        lambda x: processing_date_org(x, columns), axis=1, result_type="expand"
    )

100%|██████████| 40/40 [00:29<00:00,  1.38it/s]


### Выделение уровнею бюджета

In [23]:
list_local = ["муниципальный уровень", "местный бюджет"]
list_sub = [
    "уровень субъекта рф",
    "бюджет субъекта российской федерации",
    "бюджет территориального государственного внебюджетного фонда",
    "бюджет территориального государственного внебюджетного фонда",
]
list_fed = [
    "федеральный уровень",
    "федеральный бюджет",
    "бюджет пенсионного фонда российской федерации",
    "бюджет федерального фонда обязательного медицинского страхования",
    "бюджет фонда социального страхования российской федерации",
]

list_fed_2 = [
    "войскавая",
    "войсковая",
    "воениз",
    "федеральн",
    "район водных путей и судоходства",
    "следвест",
    "пенсион",
    "росган",
    "фгбу",
    "всероссийс",
    " фбу",
    "прокурату",
    "университ",
    "научно-исследователь",
    "государственное научное учреждение",
    "росграниц",
    "российской академии наук",
    "внутренних дел",
]
list_local_2 = [
    "муниципал",
    "школа",
    "детский сад",
    "городского поселения",
    "городского округа",
    "администрация рабочего поселка",
]
list_sub_2 = [
    "центр занятости населения",
    "област",
    "республи",
    "края",
    "край",
    "краев",
    "города",
    "автоном",
    "oбластное",
    "здравоохране",
    "больниц",
    "родильный дом",
    "профессиональн",
    "детский дом",
    "дом-интернат",
    "социальн",
    "поликлиник",
    "больниц",
    "государственное бюджетное общеобразовательное учреждение",
    "государственное бюджетное образовательное учреждение",
    "медико-санитарная часть",
    "московское государственное унитарное предприятие",
    "учреждение культуры города",
    "государственное бюджетное учреждение культуры",
    "центр социального обслуживания",
    "социального обслуживания граждан",
    "стоматологическая поликлиника",
    "центр для детей-сирот и детей",
    "санкт-петербургс",
    "фонд социального страхования российской федерации",
    "государственное бюджетное учреждение",
    "инспекция труда",
]
list_anothe = [
    "акционерное общество",
    "завод",
    "акционерное московское общество",
    "общество с ограниченной ответственностью",
]
list_fed_3 = []
list_sub_3 = [
    "ветеринар",
]
list_local_3 = ["района", "сельск"]

inn_sub = {
    "7727795994": 'Государственное бюджетное научное учреждение "Московский институт развития образования"',
    "4205050521": 'государственное учреждение "Кузбасспассажиравтотранс"',
}
inn_fed = {"7704193182": "Региональное оперативно-поисковое управление"}
inn_mun = {}

name_for_result = ["местный", "субъектовый", "федеральный", "иное"]

def fillna_budget_level(budget_level: str, full_name_customer: str, inn_customer: str):
    for list_name, name in zip(
        [list_local, list_sub, list_fed], ["местный", "субъектовый", "федеральный"]
    ):
        for name_trigger in list_name:
            if name_trigger.lower() in budget_level:
                return name

    # если не получилось выделить данные из budget_level попробуем сделать это с full_name_customer
    for list_name, name in zip(
        [list_fed_2, list_local_2, list_sub_2, list_anothe],
        ["федеральный", "местный", "субъектовый", "иное"],
    ):
        for name_trigger in list_name:
            if name_trigger.lower() in full_name_customer:
                return name

    for list_name, name in zip(
        [list_fed_3, list_local_3, list_sub_3], ["федеральный", "местный", "субъектовый", "иное"]
    ):
        for name_trigger in list_name:
            if name_trigger.lower() in full_name_customer:
                return name

    for inn_dict, name in zip(
        [inn_mun, inn_sub, inn_fed], ["местный", "субъектовый", "федеральный"]
    ):
        for inn in inn_dict.keys():
            if inn == inn_customer:
                return name

    return np.nan

In [ ]:
def processing_date_org(df: Series, columns: list):
    dict_result = {}
    # если мы получает nan при обращении к дате в DataFrame, то
    # np.nan == nan возращает False, однако у nan type float, когда у всех дат str   
    uniq_number = df["code"]

    date_registration = df["date_registration"] if type(df["date_registration"]) == str else None
    date_last_change = df["date_last_change"] if type(df["date_last_change"]) == str else None
    date_registration_tax = (
        df["date_registration_tax"] if type(df["date_registration_tax"]) == str else None
    )
    date_iky = df["date_iky"] if type(df["date_iky"]) == str else None

    dict_result["date_registration"] = date_extract(date_registration, uniq_number)
    dict_result["date_last_change"] = date_extract(date_last_change, uniq_number)
    dict_result["date_registration_tax"] = date_extract(date_registration_tax, uniq_number)
    #dict_result["date_iky"] = date_extract(date_iky, uniq_number)
    
    # определяем уровень бюджета
    budget_level = str(df["budget_level"]).lower() if type(df["budget_level"]) == str else None
    full_name_customer = str(df["full_name_customer"]).lower()
    inn_customer = str(df["inn_customer"])
    budget_level = fillna_budget_level(budget_level, full_name_customer, unique_code)
    
    

    return pd.Series(dict_result)[columns].to_list()

## Обработка данных по контрактам

In [94]:
df = pd.read_csv("../data/raw_data/contract/2014/0.csv", sep="|", dtype="str")

In [95]:
df.columns

Index(['number_contract', 'adress_customer', 'full_name_customer',
       'short_name_customer', 'unique_site_code', 'unique_site_id',
       'id_customer', 'inn_customer', 'kpp_customer', 'code_form_org',
       'okpo_code', 'municipal_code', 'budget_name', 'extrabudget_name',
       'budget_level', 'contract_status', 'notice', 'ikz_code',
       'id_contract_electronic', 'unique_number_plan',
       'method_determinig_supplier', 'date_summarizing', 'date_posting',
       'grouds_single_supplier', 'document_details', 'info_support',
       'find_date_contract', 'date_performance', 'date_contract_registry',
       'date_update_registry', 'date_start_performance',
       'date_end_performance', 'contract_item', 'contract_price',
       'contract_price_nds', 'prepayment_amount', 'performance_security',
       'size_performance_quality', 'warranty_period', 'place_performance',
       'full_name_supplier', 'inn_supplier', 'kpp_supplier',
       'code_okpo_supplier', 'date_registration_supp

In [96]:
df.adress_supplier

0                                                                                      г.Смоленск, Тульский переулок, 7
1                                                                          214019, город Смоленск, ул. Тенишевой, д. 33
2                                                           214013, Смоленская область, г. Смоленск, Тульский пер. д. 7
3                                       216200, Смоленская область, Духовщинский район, д.Заберезье, ул.Лукшинская, д.6
4                                                                                     216410 п.Шумячи,ул.Заводская д.25
                                                             ...                                                       
5568                                               Россия, 394038, Воронежская обл., Воронеж г., Пеше-Стрелецкая, д. 74
5569                               Россия, 344018, Ростовская обл., Ростов-на-Дону г., -, Буденновский пр-кт., д. 80 ,4
5570    422730, Российская Федерация, ре

### Дополнение уровней бюджета

In [2]:
list_local = ["муниципальный уровень", "местный бюджет"]
list_sub = [
    "уровень субъекта рф",
    "бюджет субъекта российской федерации",
    "бюджет территориального государственного внебюджетного фонда",
    "бюджет территориального государственного внебюджетного фонда",
]
list_fed = [
    "федеральный уровень",
    "федеральный бюджет",
    "бюджет пенсионного фонда российской федерации",
    "бюджет федерального фонда обязательного медицинского страхования",
    "бюджет фонда социального страхования российской федерации",
]

list_fed_2 = [
    "войскавая",
    "войсковая",
    "воениз",
    "федеральн",
    "район водных путей и судоходства",
    "следвест",
    "пенсион",
    "росган",
    "фгбу",
    "всероссийс",
    " фбу",
    "прокурату",
    "университ",
    "научно-исследователь",
    "государственное научное учреждение",
    "росграниц",
    "российской академии наук",
    "внутренних дел",
]
list_local_2 = [
    "муниципал",
    "школа",
    "детский сад",
    "городского поселения",
    "городского округа",
    "администрация рабочего поселка",
]
list_sub_2 = [
    "центр занятости населения",
    "област",
    "республи",
    "края",
    "край",
    "краев",
    "города",
    "автоном",
    "oбластное",
    "здравоохране",
    "больниц",
    "родильный дом",
    "профессиональн",
    "детский дом",
    "дом-интернат",
    "социальн",
    "поликлиник",
    "больниц",
    "государственное бюджетное общеобразовательное учреждение",
    "государственное бюджетное образовательное учреждение",
    "медико-санитарная часть",
    "московское государственное унитарное предприятие",
    "учреждение культуры города",
    "государственное бюджетное учреждение культуры",
    "центр социального обслуживания",
    "социального обслуживания граждан",
    "стоматологическая поликлиника",
    "центр для детей-сирот и детей",
    "санкт-петербургс",
    "фонд социального страхования российской федерации",
    "государственное бюджетное учреждение",
    "инспекция труда",
]
list_anothe = [
    "акционерное общество",
    "завод",
    "акционерное московское общество",
    "общество с ограниченной ответственностью",
]
list_fed_3 = []
list_sub_3 = [
    "ветеринар",
]
list_local_3 = ["района", "сельск"]

inn_sub = {
    "7727795994": 'Государственное бюджетное научное учреждение "Московский институт развития образования"',
    "4205050521": 'государственное учреждение "Кузбасспассажиравтотранс"',
}
inn_fed = {"7704193182": "Региональное оперативно-поисковое управление"}
inn_mun = {}

name_for_result = ["местный", "субъектовый", "федеральный", "иное"]

def fillna_budget_level(budget_level: str, full_name_customer: str, inn_customer: str):
    for list_name, name in zip(
        [list_local, list_sub, list_fed], ["местный", "субъектовый", "федеральный"]
    ):
        for name_trigger in list_name:
            if name_trigger.lower() in budget_level:
                return name

    # если не получилось выделить данные из budget_level попробуем сделать это с full_name_customer
    for list_name, name in zip(
        [list_fed_2, list_local_2, list_sub_2, list_anothe],
        ["федеральный", "местный", "субъектовый", "иное"],
    ):
        for name_trigger in list_name:
            if name_trigger.lower() in full_name_customer:
                return name

    for list_name, name in zip(
        [list_fed_3, list_local_3, list_sub_3], ["федеральный", "местный", "субъектовый", "иное"]
    ):
        for name_trigger in list_name:
            if name_trigger.lower() in full_name_customer:
                return name

    for inn_dict, name in zip(
        [inn_mun, inn_sub, inn_fed], ["местный", "субъектовый", "федеральный"]
    ):
        for inn in inn_dict.keys():
            if inn == inn_customer:
                return name

    return np.nan

In [4]:
def processing_budget(X: Series):
    budget_level = str(X["budget_level"]).lower()
    full_name_customer = str(X["full_name_customer"]).lower()
    inn_customer = str(X["inn_customer"])

    budget_level = fillna_budget_level(budget_level, full_name_customer, inn_customer)

    return budget_level

In [ ]:
df = pd.read_csv("../data/raw_data/contract/2014/37.csv", sep="|", dtype="str")
df_test = df.copy()
df_test["budget_level_new"] = df_test.apply(processing_budget, axis=1)
print("Количество прокусков:", df_test["budget_level_new"].isna().sum())
df_test["budget_level_new"].value_counts()

In [ ]:
df_test.loc[df_test.budget_level_new.isna(), "full_name_customer"].unique()

In [ ]:
df_test.loc[
    df_test.full_name_customer == 'государственное учреждение "Кузбасспассажиравтотранс"',
    "inn_customer",
].unique()

In [ ]:
## проверка на других данных
path_test = "../data/raw_data/contract/2014/"
list_problem = []
list_fullname_problem = []
for file_name in tqdm(
    sorted(os.listdir(path_test), key=lambda x: int(x.removesuffix(".csv")))[200:]
):
    df_test_now = pd.read_csv(os.path.join(path_test, file_name), sep="|", dtype="str")
    df_test_now["budget_level_new"] = df_test_now.parallel_apply(processing_budget, axis=1)
    if df_test["budget_level_new"].isna().sum() == 0:
        continue
    else:
        list_problem.append((file_name, df_test["budget_level_new"].isnull().sum()))
        list_fullname_problem.extend(
            list(df_test.loc[df_test.budget_level_new.isna(), "full_name_customer"].unique())
        )

## Обработка дат

In [5]:
df = pd.read_csv("../data/raw_data/contract/2014/37.csv", sep="|", dtype="str")

In [6]:
dates = [i for i in df.columns if "date" in i]
dates

['date_summarizing',
 'date_posting',
 'find_date_contract',
 'date_performance',
 'date_contract_registry',
 'date_update_registry',
 'date_start_performance',
 'date_end_performance',
 'date_registration_supplier']

In [7]:
date = df.date_performance.iloc[0]

In [8]:
qw = df.date_performance.loc[0]

In [11]:
def make_date(df: Series):
    # если мы получает nan при обращении к дате в DataFrame, то
    # np.nan == nan влзращает False, однако у nan type float, когда у всех дат str
    number_contract = df["number_contract"]

    date_summarizing = df["date_summarizing"] if type(df["date_summarizing"]) == str else None
    date_posting = df["date_posting"] if type(df["date_posting"]) == str else None
    date_contract = df["find_date_contract"] if type(df["find_date_contract"]) == str else None
    date_performance = df["date_performance"] if type(df["date_performance"]) == str else None
    date_contract_registry = (
        df["date_contract_registry"] if type(df["date_contract_registry"]) == str else None
    )
    date_update_registry = (
        df["date_update_registry"] if type(df["date_update_registry"]) == str else None
    )
    date_start_performance = (
        df["date_start_performance"] if type(df["date_start_performance"]) == str else None
    )
    date_end_performance = (
        df["date_end_performance"] if type(df["date_end_performance"]) == str else None
    )
    date_registration_supplier = (
        df["date_registration_supplier"]
        if type(df["date_registration_supplier"]) == str
        else None
    )

    if not date_start_performance:
        date_start_performance = date_contract

    date_summarizing = date_extract(date_summarizing, number_contract)
    date_posting = date_extract(date_posting, number_contract)
    date_contract = date_extract(date_contract, number_contract)
    date_performance = date_extract(date_performance, number_contract)
    date_contract_registry = date_extract(date_contract_registry, number_contract)
    date_update_registry = date_extract(date_update_registry, number_contract)
    date_start_performance = date_extract(date_start_performance, number_contract)
    date_end_performance = date_extract(date_end_performance, number_contract)
    date_registration_supplier = date_extract(date_registration_supplier, number_contract)

    return (
        date_summarizing,
        date_posting,
        date_contract,
        date_performance,
        date_contract_registry,
        date_update_registry,
        date_start_performance,
        date_end_performance,
        date_registration_supplier,
    )

In [22]:
df_copy = df.copy()

In [23]:
df_copy[
    [
        "date_summarizing_new",
        "date_posting_new",
        "find_date_contract_new",
        "date_performance_new",
        "date_contract_registry_new",
        "date_update_registry_new",
        "date_start_performance_new",
        "date_end_performance_new",
        "date_registration_supplier_new",
    ]
] = df_copy.apply(make_date, axis=1, result_type="expand")

In [24]:
for i in [
    "date_summarizing_new",
    "date_posting_new",
    "find_date_contract_new",
    "date_performance_new",
    "date_contract_registry_new",
    "date_update_registry_new",
    "date_start_performance_new",
    "date_end_performance_new",
    "date_registration_supplier_new",
]:
    print(df_copy[[i.removesuffix("_new"), i]].isnull().sum())

date_summarizing        1143
date_summarizing_new    1143
dtype: int64
date_posting        0
date_posting_new    0
dtype: int64
find_date_contract        0
find_date_contract_new    0
dtype: int64
date_performance        0
date_performance_new    0
dtype: int64
date_contract_registry        0
date_contract_registry_new    0
dtype: int64
date_update_registry        0
date_update_registry_new    0
dtype: int64
date_start_performance        5357
date_start_performance_new    5357
dtype: int64
date_end_performance        0
date_end_performance_new    0
dtype: int64
date_registration_supplier        5363
date_registration_supplier_new    5363
dtype: int64


In [25]:
qw = "date_summarizing_new"
df_copy[[qw.removesuffix("_new"), qw]]

,date_summarizing,date_summarizing_new
0,28.03.2014,2014-03-28
1,18.03.2014,2014-03-18
2,28.03.2014,2014-03-28
3,27.03.2014,2014-03-27
4,28.03.2014,2014-03-28
...,...,...
5557,NaN,None
5558,02.04.2014,2014-04-02
5559,NaN,None
5560,NaN,None


In [26]:
path_test = "../data/raw_data/contract/2014/"
for file_name in tqdm(sorted(os.listdir(path_test), key=lambda x: int(x.removesuffix(".csv")))):
    df_test_now = pd.read_csv(os.path.join(path_test, file_name), sep="|", dtype="str")
    df_test_now[
        [
            "date_summarizing_new",
            "date_posting_new",
            "find_date_contract_new",
            "date_performance_new",
            "date_contract_registry_new",
            "date_update_registry_new",
            "date_start_performance_new",
            "date_end_performance_new",
            "date_registration_supplier_new",
        ]
    ] = df_test_now.parallel_apply(make_date, axis=1, result_type="expand")

  6%|▋         | 32/500 [00:20<04:54,  1.59it/s]

0307300000714000007
Дата окончания исполнения контракта
['Дата окончания исполнения контракта']


 12%|█▏        | 60/500 [00:38<04:34,  1.61it/s]

0319300086014000023
Дата окончания исполнения контракта
['Дата окончания исполнения контракта']


 25%|██▍       | 124/500 [01:20<03:52,  1.62it/s]

...

 97%|█████████▋| 485/500 [05:12<00:09,  1.58it/s]

0318300031214000010
Дата окончания исполнения контракта
['Дата окончания исполнения контракта']


 98%|█████████▊| 490/500 [05:15<00:06,  1.59it/s]

0301300442814000007
Дата окончания исполнения контракта
['Дата окончания исполнения контракта']


100%|██████████| 500/500 [05:21<00:00,  1.55it/s]


## Расшифровка КБК

In [12]:
kbk_type = pd.read_excel("../data/kbk.xlsx", sheet_name="type", dtype="str")
kbk_np = pd.read_excel("../data/kbk.xlsx", sheet_name="np", dtype="str")
kbk_section = pd.read_excel("../data/kbk.xlsx", sheet_name="section", dtype="str")

In [13]:
df = pd.read_csv("../data/raw_data/contract/2014/0.csv", sep="|", dtype="str")
kbk = df.loc[3, "kbk"]

In [18]:
def extract_data_from_kbk(kbk, year):
    dict_kbk = {
        "code_main_admin": None,
        "code_section_sub": None,
        "code_direction_expenses": None,
        "code_type_expenses": None,
        "code_national_project": None,
        "value_code_section": None,
        "value_code_sub": None,
        "value_code_type_expenses": None,
        "name_national_project": None,
        "name_fed_national_project": None,
    }
    if not kbk:
        return dict_kbk

    if len(kbk) == 3 or kbk[:-3] == "0" * 17:
        code_type_expenses = kbk[-3:]
        value_code_type_expenses = kbk_type.loc[
            kbk_type.code == code_type_expenses, "mean"
        ].to_list()

        if len(value_code_type_expenses):
            dict_kbk["value_code_type_expenses"] = value_code_type_expenses[0]
        else:
            pass
            # логи
        dict_kbk["code_type_expenses"] = code_type_expenses
        return dict_kbk

    elif len(kbk) == 20:
        kbk_search = re.compile(r"(\S\S\S)(\S\S\S\S)(\S\S\S\S\S\S\S\S\S\S)(\S\S\S)")
        kbk_find = kbk_search.search(kbk)

        # код главного распоредителя бюджетных средств
        code_main_admin = kbk_find.group(1)
        # print('Код ГРС:', code_main_admin)
        # код раздела и подраздела
        code_section_sub = kbk_find.group(2)
        # print('Код раздела и подраздела:', code_section_sub)
        # код целевой статьи
        code_direction_expenses = kbk_find.group(3)
        # print('Код целевой статьи:', code_direction_expenses)
        # код вида расходов
        code_type_expenses = kbk_find.group(4)
        # print('Код вида расходов:', code_type_expenses)
        # код национального проекта
        code_national_project = (
            code_direction_expenses[3:5] if not code_direction_expenses[3].isdigit() else None
        )
        # print('Код национального проекта:', code_national_project)

        value_code_section = kbk_section.loc[
            (kbk_section.year == year) & (kbk_section.code == code_section_sub[:2]), "mean"
        ].to_list()
        if len(value_code_section):
            dict_kbk["value_code_section"] = value_code_section[0]
        else:
            pass
        # print('value_code_section:', value_code_section)

        value_code_sub = kbk_section.loc[
            (kbk_section.year == year) & (kbk_section.code == code_section_sub), "mean"
        ].to_list()
        if len(value_code_sub):
            dict_kbk["value_code_sub"] = value_code_sub[0]
        else:
            pass
        # print('code_type_expenses:', value_code_sub)

        value_code_type_expenses = kbk_type.loc[
            kbk_type.code == code_type_expenses, "mean"
        ].to_list()
        if len(value_code_type_expenses):
            dict_kbk["value_code_type_expenses"] = value_code_type_expenses[0]
        else:
            pass

        # print('value_code_type_expenses:', value_code_type_expenses)
        if code_national_project:
            list_national_project = kbk_np.loc[
                (kbk_np.year == year) & (kbk_np.code == code_national_project),
                ["name_national_project", "name_fed_national_project"],
            ].values
            # print('list_national_project:', list_national_project)

            if len(list_national_project):
                dict_kbk["name_national_project"] = list_national_project[0]
                dict_kbk["name_fed_national_project"] = list_national_project[1]
            else:
                pass
                # логи

        dict_kbk["code_main_admin"] = code_main_admin
        dict_kbk["code_section_sub"] = code_section_sub
        dict_kbk["code_direction_expenses"] = code_direction_expenses
        dict_kbk["code_type_expenses"] = code_type_expenses
        dict_kbk["code_national_project"] = code_national_project

        return dict_kbk

    else:
        # print(kbk)
        return dict_kbk
        # добавить логи

In [19]:
list_p = []

if len(list_p):
    print("owl")

In [20]:
def make_date(df: Series, columns: list):
    dict_result = {}
    # если мы получает nan при обращении к дате в DataFrame, то
    # np.nan == nan влзращает False, однако у nan type float, когда у всех дат str
    number_contract = df["number_contract"]

    date_summarizing = df["date_summarizing"] if type(df["date_summarizing"]) == str else None
    date_posting = df["date_posting"] if type(df["date_posting"]) == str else None
    date_contract = df["find_date_contract"] if type(df["find_date_contract"]) == str else None
    date_performance = df["date_performance"] if type(df["date_performance"]) == str else None
    date_contract_registry = (
        df["date_contract_registry"] if type(df["date_contract_registry"]) == str else None
    )
    date_update_registry = (
        df["date_update_registry"] if type(df["date_update_registry"]) == str else None
    )
    date_start_performance = (
        df["date_start_performance"] if type(df["date_start_performance"]) == str else None
    )
    date_end_performance = (
        df["date_end_performance"] if type(df["date_end_performance"]) == str else None
    )
    date_registration_supplier = (
        df["date_registration_supplier"]
        if type(df["date_registration_supplier"]) == str
        else None
    )

    if not date_start_performance:
        date_start_performance = date_contract

    dict_result["date_summarizing_new"] = date_extract(date_summarizing, number_contract)
    dict_result["date_posting_new"] = date_extract(date_posting, number_contract)
    dict_result["date_contract_new"] = date_extract(date_contract, number_contract)
    dict_result["date_performance_new"] = date_extract(date_performance, number_contract)
    dict_result["date_contract_registry_new"] = date_extract(
        date_contract_registry, number_contract
    )
    dict_result["date_update_registry_new"] = date_extract(date_update_registry, number_contract)
    dict_result["date_start_performance_new"] = date_extract(
        date_start_performance, number_contract
    )
    dict_result["date_end_performance_new"] = date_extract(date_end_performance, number_contract)
    dict_result["date_registration_supplier_new"] = date_extract(
        date_registration_supplier, number_contract
    )

    for date in [
        dict_result["date_contract_new"],
        dict_result["date_summarizing_new"],
        dict_result["date_posting_new"],
        dict_result["date_performance_new"],
        dict_result["date_end_performance_new"],
    ]:
        if type(date) == datetime.date:
            year = str(date.year)
            break

    kbk = df["kbk"] if type(df["kbk"]) == str else None

    dict_kbk_result = extract_data_from_kbk(kbk, year)

    dict_result.update(dict_kbk_result)

    # print(df.name)

    return pd.Series(dict_result)[columns].to_list()

In [21]:
df_copy = df.copy()

In [22]:
columns = [
    "date_summarizing_new",
    "date_posting_new",
    "date_contract_new",
    "date_performance_new",
    "date_contract_registry_new",
    "date_update_registry_new",
    "date_start_performance_new",
    "date_end_performance_new",
    "date_registration_supplier_new",
    "code_main_admin",
    "code_section_sub",
    "code_direction_expenses",
    "code_type_expenses",
    "code_national_project",
    "value_code_section",
    "value_code_sub",
    "value_code_type_expenses",
    "name_national_project",
    "name_fed_national_project",
]
df_copy[columns] = df_copy.apply(
    lambda x: make_date(x, columns=columns), axis=1, result_type="expand"
)

In [27]:
df.columns

Index(['number_contract', 'adress_customer', 'full_name_customer',
       'short_name_customer', 'unique_site_code', 'unique_site_id',
       'id_customer', 'inn_customer', 'kpp_customer', 'code_form_org',
       'okpo_code', 'municipal_code', 'budget_name', 'extrabudget_name',
       'budget_level', 'contract_status', 'notice', 'ikz_code',
       'id_contract_electronic', 'unique_number_plan',
       'method_determinig_supplier', 'date_summarizing', 'date_posting',
       'grouds_single_supplier', 'document_details', 'info_support',
       'find_date_contract', 'date_performance', 'date_contract_registry',
       'date_update_registry', 'date_start_performance',
       'date_end_performance', 'contract_item', 'contract_price',
       'contract_price_nds', 'prepayment_amount', 'performance_security',
       'size_performance_quality', 'warranty_period', 'place_performance',
       'full_name_supplier', 'inn_supplier', 'kpp_supplier',
       'code_okpo_supplier', 'date_registration_supp

## Выявление плохих адресов

In [29]:
df = pd.read_csv("../data/raw_data/contract/2014/0.csv", sep="|", dtype="str")

In [51]:
df.shape

(5573, 52)

In [32]:
df.adress_customer.isnull().sum()

208

Обработать: "Юридический адрес: Российская Федерация, 660049, Красноярский край, Красноярск г, Мира, 76, -
Фактический адрес: Российская Федерация, 660062, Красноярский край, Красноярск г, Высотная, 9"

hodashooh@mail.ru
ulibka6@mail.ru
detskisad27@mail.ru
rucheekdou@mail.ru
zelnurlat@mail.ru
balashova.liudmila2013@yandex.ru
topolek51@mail.ru
shirdan@yandex.ru
""

Посмотреть как с такими адрессами будет работать pulenti

Красная строка в адрессе:  
0352200002914000002 0158200002314000019 0372200037314000013

In [75]:
df = pd.read_csv("../data/contract_number/numbers/2014.csv", sep="|", dtype="str")

In [76]:
index_telephon = df.adress_customer.str.replace("-", "").str.isdigit().fillna(False)
index_email = df.adress_customer.str.find("@") > -1
index_isull = df.adress_customer.isnull()

In [80]:
bad_index = index_telephon | index_email | index_isull

In [ ]:
data_org = pd.DataFrame()


def check_adress(adress, code, code_type):
    list_check = ["Российская Федерация", "РФ", "обл", "ул", "край", "г,", "п."]
    is_nan = type(adress) == float
    is_telephon = adress.replace("-", "").isdigit()
    is_email = ("@" in adress) and not any([i in adress for i in list_check])

    need_replace = any([is_nan, is_telephon, is_email])

    if not need_replace:
        return adress
    else:
        adress = data_org.loc[
            (data_org.code == code) & (data_org.code_type == code_type), "adress"
        ]
        return adress